In [ ]:
!pip install -U word_forms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 166 kB 4.2 MB/s 
  Attempting uninstall: inflect
    Found existing installation: inflect 2.1.0
    Uninstalling inflect-2.1.0:
      Successfully uninstalled inflect-2.1.0


In [ ]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch
import torchtext

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/ProgettiDataScience-SAD/1 ANNO 2 SEMESTRE/DATA SEMANTICS/Progetto/Newspaper')

In [ ]:
# !pip install glove_python
# !pip install glove-py
# !pip install glove_pybind
!pip install glove-python-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 3.7 MB/s 


# Pre 2001

## Preprocessing

In [ ]:
from word_forms.word_forms import get_word_forms
import nltk
nltk.download('omw-1.4')

# generate all possible forms of keywords
# keyword_list = ['bin laden', 'al qaeda', 'world trade center', 'twin towers', 'attack', 'hijack', 'firefighter', 'plane', 'terrorism', 'afghanistan', 'jihad', 'islam']
word_form_tower = get_word_forms("tower")
word_form_attack = get_word_forms("attack")
word_form_hijack = get_word_forms("hijack")
word_form_firefighter = get_word_forms("firefighter")
word_form_plane = get_word_forms("plane")
word_form_terrorism = get_word_forms("terrorism")


word_form_tot = (list(list(word_form_tower['a']) + list(word_form_tower['n']) + 
list(word_form_tower['v']) + list(word_form_tower['r']) + 
list(word_form_attack['a']) + list(word_form_attack['n']) + 
list(word_form_attack['v']) + list(word_form_attack['r']) + 
list(word_form_plane['a']) + list(word_form_plane['n']) + 
list(word_form_plane['v']) + list(word_form_plane['r']) + 
list(word_form_hijack['a']) + list(word_form_hijack['n']) + 
list(word_form_hijack['v']) + list(word_form_hijack['r']) + 
list(word_form_firefighter['a']) + list(word_form_firefighter['n']) + 
list(word_form_firefighter['v']) + list(word_form_firefighter['r']) + 
list(word_form_terrorism['a']) + list(word_form_terrorism['n']) + 
list(word_form_terrorism['v']) + list(word_form_terrorism['r'])))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# import string
import re

BASE_PATH = '/content/drive/MyDrive/ProgettiDataScience-SAD/1 ANNO 2 SEMESTRE/DATA SEMANTICS/Progetto/Newspaper/'


with open(BASE_PATH + 'text_news_nne/text_news_1996.txt') as read_file1:
  sentences1996 = read_file1.readlines()
with open(BASE_PATH + 'text_news_nne/text_news_1997.txt') as read_file2:
  sentences1997 = read_file2.readlines()
with open(BASE_PATH + 'text_news_nne/text_news_1998.txt') as read_file3:
  sentences1998 = read_file3.readlines()
with open(BASE_PATH + 'text_news_nne/text_news_1999.txt') as read_file4:
  sentences1999 = read_file4.readlines()
with open(BASE_PATH + 'text_news_nne/text_news_2000.txt') as read_file5:
  sentences2000 = read_file5.readlines()

sentences9600 = sentences1996 + sentences1997 + sentences1998 + sentences1999 + sentences2000

  
#define the list of words of our interest
keywords = ['bin laden', 'al qaeda', 'twin towers', 'world trade center', 'afghanistan', 'jihad', 'islam'] #aggiungere twin towers

keyword_list = keywords + word_form_tot

#define the dictionary where we store the articles
articles = {}
art = {}
#remove the p that represent in html the start of the paragraph
for symbol in sentences9600:
  symbol = symbol.replace('<p>', '')
  #selection of articles that contain the keywords
  for keyword in keyword_list:
    if (keyword in symbol.lower()):
      #select id of the article that contains at least one kwyword 
      id = re.findall(r'@@(\w+)', symbol)[0]
      articles[id] = symbol.lower()
      art[id] = symbol.lower()

print(len(articles))

7958


In [ ]:
import spacy
sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words

cont = 0
support_dict = {}

for id in art:
  #tokenization
  token_sentences9600_art = art[id].split()

  #stop-word removal
  stop_word_art = [i for i in token_sentences9600_art if str(i).lower() not in all_stopwords]
  words_art = [i for i in stop_word_art if i.isalpha() or i == "."]
  
  art[id] = words_art

  # bigrams = list(nltk.ngrams(words_art,2))
  # trigrams = list(nltk.ngrams(words_art,3))
  # n_most_common = 0 #how many of the most common n-grams do you want, zero indicates only the user_grams.
  # fdist_bigrams = nltk.FreqDist(bigrams).most_common(n_most_common) #n most common bigrams
  # fdist_trigrams = nltk.FreqDist(trigrams).most_common(n_most_common) #n most common trigrams
  # fdist_bigrams=[x[0][0]+' '+x[0][1] for x in fdist_bigrams]
  # fdist_trigrams=[x[0][0]+' '+x[0][1]+' '+x[0][2] for x in fdist_trigrams]
  # n_grams=fdist_bigrams
  # n_grams.extend(fdist_trigrams)

  user_grams = ['bin laden', 'al qaeda', 'world trade center', 'twin towers', 'white house', 'president bush', 'bay area']

  i=0
  support_dict[id] = []
  #check if 
  # - word in position i + space + word in position i+1 + space + word in position i+2 is a word in user_grams (world trade center)
  # - word in position i + space + word in position i+1 + space is a word in user_grams (ex. bin + ' ' + laden)
  # and create a unique term in the vocabulary (ex. bin-laden)
  # otherwise leave the single word 
  while i < len(art[id])-1:
    try:
      if (art[id][i] + ' ' + art[id][i+1] + ' ' + art[id][i+2] in user_grams):
        support_dict[id].append(art[id][i] + '-' + art[id][i+1] + '-' + art[id][i+2])
        i = i + 3
      elif(art[id][i] + ' ' + art[id][i+1] in user_grams):
        support_dict[id].append(art[id][i] + '-' + art[id][i+1])
        i = i + 2
      else:
        support_dict[id].append(art[id][i])
        i = i + 1
    except:
      if(art[id][i] + ' ' + art[id][i+1] in user_grams):
        support_dict[id].append(art[id][i] + '-' + art[id][i+1])
        i = i + 2
      else:
        support_dict[id].append(art[id][i])
        i = i + 1
      pass
    # print(i, 'out of', len(art[id]))

# just to know how many articles have been checked
  cont = cont + 1
  # print(cont)

In [ ]:
def splitt(sequence, sep):
    chunk = []
    for val in sequence:
        if val == sep:
            yield chunk
            chunk = []
        else:
            chunk.append(val)
    yield chunk

In [ ]:
# support_dict['3049580']

In [ ]:
# res = splitt(support_dict['3049580'], '.')

In [ ]:
# for x in res:
#   print(x)

In [ ]:
listGloVe9600 = []
for id in support_dict:
  res = splitt(support_dict[id], '.')
  for x in res:
    listGloVe9600.append(x)
  # listGloVe9600.remove([])

print(len(listGloVe9600))

748291


## GloVe

In [ ]:
from glove import Glove, Corpus
# from glove import *

# from glove_pybind import *
# from Glove import *


#Creating a corpus object
corpus = Corpus() # dictionary = dictionary2001 doesn't work

# lines = []
# for i in range(0, 1475):
#   lines.append(t[i][1])


#Training the corpus to generate the co-occurrence matrix which is used in GloVe
corpus.fit(listGloVe9600, window=3)

glove9600 = Glove() 
glove9600.fit(corpus.matrix, epochs=40, no_threads=4, verbose=True)
glove9600.add_dictionary(corpus.dictionary)
glove9600.save('glove9600.model')

Performing 40 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39


### Most similar to

In [ ]:
# glove9600.most_similar("al-qaeda", number= 10)

In [ ]:
glove9600.most_similar("bin-laden", number= 10)

[('osama', 0.844677878355939),
 ('bandaged', 0.7005026174706804),
 ('kraar', 0.688719919721171),
 ('lymon', 0.6882155138646375),
 ('bradens', 0.6856981891133778),
 ('relational', 0.6832710438168057),
 ('rotimi', 0.6814091230214151),
 ('komaiko', 0.6807929649503708),
 ('ivs', 0.6783447408652903)]

In [ ]:
glove9600.most_similar("airplane", number= 10)

[('astral', 0.7880179596606505),
 ('fly', 0.7865634752896001),
 ('plane', 0.785164384741643),
 ('planes', 0.7543480595372443),
 ('passengers', 0.7064090357229778),
 ('flights', 0.7055839002647677),
 ('passenger', 0.6968535496427386),
 ('flying', 0.6948862363786642),
 ('crashes', 0.6821687568994562)]

In [ ]:
glove9600.most_similar("twin-towers", number= 10)

[('whopping', 0.744859732602848),
 ('tbsps', 0.7323463909186141),
 ('rapes', 0.7232844130830011),
 ('arrestees', 0.7169614957106304),
 ('amounted', 0.7141235229665089),
 ('plus', 0.7128216557980696),
 ('ninety', 0.710433856623177),
 ('minus', 0.697677961828906),
 ('amassed', 0.6941959607927356)]

In [ ]:
glove9600.most_similar("tower", number= 10)

[('eiffel', 0.8887910748063228),
 ('sewer', 0.6560821038511232),
 ('sears', 0.6555181653030608),
 ('pelt', 0.6501763684509091),
 ('inside', 0.6476980443599056),
 ('blueline', 0.6377410673056577),
 ('chelly', 0.6326826602261479),
 ('leinsdorf', 0.6303701744851642),
 ('coit', 0.629017064464484)]

In [ ]:
glove9600.most_similar("terrorism", number= 10)

[('faction', 0.713116548061407),
 ('filmmaking', 0.7005818715073955),
 ('peloponnesian', 0.6920328477554689),
 ('waging', 0.6798692065040008),
 ('supplier', 0.6750925942001275),
 ('unrest', 0.6687576257430208),
 ('domestic', 0.6550042235270525),
 ('waged', 0.6496534189943141),
 ('maneuverings', 0.6353513303194234)]

In [ ]:
glove9600.most_similar("world-trade-center", number= 10)

[('bombing', 0.7665277311112741),
 ('primaries', 0.7180936775309205),
 ('presided', 0.7159578397476346),
 ('airstrikes', 0.6894978160620931),
 ('kibu', 0.6842326509372547),
 ('nairobi', 0.6829136741582353),
 ('lyric', 0.6794456929100989),
 ('minot', 0.6785776738283376),
 ('suthun', 0.6768834757189469)]

In [ ]:
glove9600.most_similar("attack", number= 10)

[('heart', 0.894287783686664),
 ('attacks', 0.8610937059746007),
 ('palpitations', 0.7464569750621054),
 ('abiomed', 0.7369885198720039),
 ('terrorist', 0.7290647434540147),
 ('failure', 0.7138972064566119),
 ('congestive', 0.6936525587083107),
 ('artificial', 0.6907880266950351),
 ('disease', 0.6680527156805398)]

In [ ]:
glove9600.most_similar("hijacked", number= 10)

# 'hijacker',
#  'hijackings',
#  'hijackers',
#  'hijack',
#  'hijacks',
#  'hijacking',
#  'hijacked',
#  'hijack',
#  'hijacks',
#  'hijacking'

[('downed', 0.7648946089214631),
 ('piloting', 0.7622181893557346),
 ('intoxicated', 0.7326077174548769),
 ('inbound', 0.7310467214461046),
 ('astral', 0.7164958951066336),
 ('micromanagement', 0.7095074216344937),
 ('tote', 0.7023444340592163),
 ('plane', 0.7022806032925448),
 ('equator', 0.6975287189412865)]

In [ ]:
glove9600.most_similar("firefighter", number= 10)

[('blackhawks', 0.7149312418192217),
 ('hilton', 0.6881183191704141),
 ('towers', 0.680300130416833),
 ('sheraton', 0.6781065472648741),
 ('lakefront', 0.6741340699998563),
 ('wls', 0.6694710532012597),
 ('regency', 0.6659115429387163),
 ('schoolteacher', 0.6567414541869231),
 ('hazelden', 0.6543817525558092)]

In [ ]:
glove9600.most_similar("afghanistan", number= 10)

[('bosnian', 0.7764144225456836),
 ('serb', 0.75460545951974),
 ('breakup', 0.713586116576805),
 ('soviet', 0.7104473006719376),
 ('taliban', 0.7074139956700906),
 ('invasion', 0.6965090480678483),
 ('spying', 0.6919502100446717),
 ('clampdown', 0.6811555339182364),
 ('tlacuitapa', 0.677420280933512)]

In [ ]:
glove9600.most_similar("islam", number= 10)

[('populous', 0.7758013787289457),
 ('christianizing', 0.7675170395652152),
 ('nation', 0.7584585551087366),
 ('sovereign', 0.7400054614495597),
 ('tigreans', 0.7377764662848282),
 ('factbook', 0.6908426178576603),
 ('poorest', 0.6687792862480656),
 ('largest', 0.6608995667104222),
 ('savannahians', 0.6570647190646466)]

In [ ]:
glove9600.most_similar("jihad", number= 10)

[('fundamentalism', 0.894466886495728),
 ('fundamentalists', 0.8791564425783244),
 ('fundamentalist', 0.8710215143165715),
 ('militant', 0.8464263200356066),
 ('radicals', 0.8408896259130032),
 ('islamic', 0.8258255961024408),
 ('militants', 0.7967244249371761),
 ('extremists', 0.7839169003348269),
 ('hamas', 0.7701835234330764)]

# 2001

## Preprocessing

In [ ]:
from word_forms.word_forms import get_word_forms
import nltk
nltk.download('omw-1.4')

# generate all possible forms of keywords
# keyword_list = ['bin laden', 'al qaeda', 'world trade center', 'twin towers', 'attack', 'hijack', 'firefighter', 'plane', 'terrorism', 'afghanistan', 'jihad', 'islam']
word_form_tower = get_word_forms("tower")
word_form_attack = get_word_forms("attack")
word_form_hijack = get_word_forms("hijack")
word_form_firefighter = get_word_forms("firefighter")
word_form_plane = get_word_forms("plane")
word_form_terrorism = get_word_forms("terrorism")


word_form_tot = (list(list(word_form_tower['a']) + list(word_form_tower['n']) + 
list(word_form_tower['v']) + list(word_form_tower['r']) + 
list(word_form_attack['a']) + list(word_form_attack['n']) + 
list(word_form_attack['v']) + list(word_form_attack['r']) + 
list(word_form_plane['a']) + list(word_form_plane['n']) + 
list(word_form_plane['v']) + list(word_form_plane['r']) + 
list(word_form_hijack['a']) + list(word_form_hijack['n']) + 
list(word_form_hijack['v']) + list(word_form_hijack['r']) + 
list(word_form_firefighter['a']) + list(word_form_firefighter['n']) + 
list(word_form_firefighter['v']) + list(word_form_firefighter['r']) + 
list(word_form_terrorism['a']) + list(word_form_terrorism['n']) + 
list(word_form_terrorism['v']) + list(word_form_terrorism['r'])))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# import string
import re

BASE_PATH = '/content/drive/MyDrive/ProgettiDataScience-SAD/1 ANNO 2 SEMESTRE/DATA SEMANTICS/Progetto/Newspaper/'


with open(BASE_PATH + 'text_news_nne/text_news_2001.txt') as read_file1:
  sentences2001 = read_file1.readlines()
  
#define the list of words of our interest
keywords = ['bin laden', 'al qaeda', 'twin towers', 'world trade center', 'afghanistan', 'jihad', 'islam'] #aggiungere twin towers

keyword_list = keywords + word_form_tot

#define the dictionary where we store the articles
articles = {}
art = {}
#remove the p that represent in html the start of the paragraph
for symbol in sentences2001:
  symbol = symbol.replace('<p>', '')
  #selection of articles that contain the keywords
  for keyword in keyword_list:
    if (keyword in symbol.lower()):
      #select id of the article that contains at least one kwyword 
      id = re.findall(r'@@(\w+)', symbol)[0]
      articles[id] = symbol.lower()
      art[id] = symbol.lower()

print(len(articles))

1558


In [ ]:
import spacy
sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words

cont = 0
support_dict = {}

for id in art:
  #tokenization
  token_sentences2001_art = art[id].split()

  #stop-word removal
  stop_word_art = [i for i in token_sentences2001_art if str(i).lower() not in all_stopwords]
  words_art = [i for i in stop_word_art if i.isalpha() or i == "."]
  
  art[id] = words_art

  # bigrams = list(nltk.ngrams(words_art,2))
  # trigrams = list(nltk.ngrams(words_art,3))
  # n_most_common = 0 #how many of the most common n-grams do you want, zero indicates only the user_grams.
  # fdist_bigrams = nltk.FreqDist(bigrams).most_common(n_most_common) #n most common bigrams
  # fdist_trigrams = nltk.FreqDist(trigrams).most_common(n_most_common) #n most common trigrams
  # fdist_bigrams=[x[0][0]+' '+x[0][1] for x in fdist_bigrams]
  # fdist_trigrams=[x[0][0]+' '+x[0][1]+' '+x[0][2] for x in fdist_trigrams]
  # n_grams=fdist_bigrams
  # n_grams.extend(fdist_trigrams)

  user_grams = ['bin laden', 'al qaeda', 'world trade center', 'twin towers', 'white house', 'president bush', 'bay area']

  i=0
  support_dict[id] = []
  #check if 
  # - word in position i + space + word in position i+1 + space + word in position i+2 is a word in user_grams (world trade center)
  # - word in position i + space + word in position i+1 + space is a word in user_grams (ex. bin + ' ' + laden)
  # and create a unique term in the vocabulary (ex. bin-laden)
  # otherwise leave the single word 
  while i < len(art[id])-1:
    try:
      if (art[id][i] + ' ' + art[id][i+1] + ' ' + art[id][i+2] in user_grams):
        support_dict[id].append(art[id][i] + '-' + art[id][i+1] + '-' + art[id][i+2])
        i = i + 3
      elif(art[id][i] + ' ' + art[id][i+1] in user_grams):
        support_dict[id].append(art[id][i] + '-' + art[id][i+1])
        i = i + 2
      else:
        support_dict[id].append(art[id][i])
        i = i + 1
    except:
      if(art[id][i] + ' ' + art[id][i+1] in user_grams):
        support_dict[id].append(art[id][i] + '-' + art[id][i+1])
        i = i + 2
      else:
        support_dict[id].append(art[id][i])
        i = i + 1
      pass
    # print(i, 'out of', len(art[id]))

# just to know how many articles have been checked
  cont = cont + 1
  # print(cont)

In [ ]:
def split(sequence, sep):
    chunk = []
    for val in sequence:
        if val == sep:
            yield chunk
            chunk = []
        else:
            chunk.append(val)
    yield chunk

In [ ]:
# support_dict['3049580']

In [ ]:
# res = split(support_dict['3049580'], '.')

In [ ]:
# for x in res:
#   print(x)

In [ ]:
listGloVe2001 = []
for id in support_dict:
  res = split(support_dict[id], '.')
  for x in res:
    listGloVe2001.append(x)
  # listGloVe2001.remove([])

print(len(listGloVe2001))

159705


In [ ]:
listGloVe2001.remove([])

## GloVe

In [ ]:
from glove import Glove, Corpus
# from glove import *

# from glove_pybind import *
# from Glove import *


#Creating a corpus object
corpus = Corpus() # dictionary = dictionary2001 doesn't work

# lines = []
# for i in range(0, 1475):
#   lines.append(t[i][1])


#Training the corpus to generate the co-occurrence matrix which is used in GloVe
corpus.fit(listGloVe2001, window=3)

glove2001 = Glove() 
glove2001.fit(corpus.matrix, epochs=40, no_threads=4, verbose=True)
glove2001.add_dictionary(corpus.dictionary)
glove2001.save('glove2001.model')

Performing 40 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39


### Most similar to

In [ ]:
glove2001.most_similar("al-qaeda", number= 10)

[('associates', 0.8592477616432557),
 ('elf', 0.8055399355606138),
 ('network', 0.7643943942700478),
 ('bin-laden', 0.7421191049046149),
 ('members', 0.736893310347024),
 ('captured', 0.7367375136319364),
 ('affiliated', 0.7334787188345392),
 ('fighters', 0.7021862385982907),
 ('disseminated', 0.6973804006302537)]

In [ ]:
glove2001.most_similar("bin-laden", number= 10)

[('osama', 0.9246469897703221),
 ('usama', 0.9139753802868363),
 ('followers', 0.8343559435478555),
 ('lieutenants', 0.8032231135016807),
 ('mastermind', 0.7782025815403385),
 ('zawahiri', 0.757665120623418),
 ('al-qaeda', 0.7421191049046149),
 ('captured', 0.7082608738501567),
 ('suspect', 0.6860534640808587)]

In [ ]:
glove2001.most_similar("airplane", number= 10)

[('commute', 0.8382529045546554),
 ('opted', 0.7610531627358545),
 ('clasped', 0.7564480057180382),
 ('caulley', 0.7557635538401715),
 ('hit', 0.7385131776651755),
 ('gangs', 0.7374209656431582),
 ('promising', 0.7164422573919873),
 ('hammered', 0.7109626217816739),
 ('warm', 0.7055146006890611)]

In [ ]:
glove2001.most_similar("twin-towers", number= 10)

[('world-trade-center', 0.9445973822437257),
 ('towers', 0.9057111486208604),
 ('attack', 0.826270152460384),
 ('imminent', 0.7884549560184695),
 ('attacks', 0.7863652135521956),
 ('pentagon', 0.7840097889594118),
 ('heart', 0.7717360219157651),
 ('devastating', 0.7524947611937014),
 ('shark', 0.7480718329232048)]

In [ ]:
glove2001.most_similar("tower", number= 10)

[('glenbrook', 0.8669747096985811),
 ('carolina', 0.8503328107191913),
 ('dakota', 0.8420217996042147),
 ('sears', 0.8361392963612109),
 ('korea', 0.8180931131580278),
 ('north', 0.8083934561628101),
 ('pole', 0.8048575121133135),
 ('slope', 0.8016276260176054),
 ('loop', 0.7978342449742244)]

In [ ]:
glove2001.most_similar("terrorism", number= 10)

[('war', 0.9419747710358454),
 ('vietnam', 0.898078796943593),
 ('hart', 0.88886019404615),
 ('cold', 0.8607994866936424),
 ('waging', 0.8550639446694331),
 ('guerrilla', 0.8518487887065184),
 ('criminals', 0.8499300578016136),
 ('ii', 0.8480210409228944),
 ('erc', 0.8392797592364825)]

In [ ]:
glove2001.most_similar("world-trade-center", number= 10)

[('twin-towers', 0.9445973822437257),
 ('towers', 0.9435471044004613),
 ('pentagon', 0.8592894219782696),
 ('collapse', 0.8153307616495667),
 ('attack', 0.7888474055697614),
 ('bombing', 0.7753251637472074),
 ('attacks', 0.7647799038600868),
 ('heart', 0.7570488976595544),
 ('collapsed', 0.7481659821596933)]

In [ ]:
glove2001.most_similar("attack", number= 10)

[('shark', 0.9158650906399304),
 ('attacks', 0.8811446935214841),
 ('heart', 0.8699385762953753),
 ('terrorist', 0.8464290920508517),
 ('twin-towers', 0.8262701524603839),
 ('devastating', 0.7995973325444293),
 ('world-trade-center', 0.7888474055697614),
 ('bioterrorist', 0.7532652357699731),
 ('occurred', 0.7462606391442814)]

In [ ]:
glove2001.most_similar("hijacked", number= 10)

# 'hijacker',
#  'hijackings',
#  'hijackers',
#  'hijack',
#  'hijacks',
#  'hijacking',
#  'hijacked',
#  'hijack',
#  'hijacks',
#  'hijacking'

[('deck', 0.8179416389717039),
 ('li', 0.8140301314624266),
 ('swissair', 0.8068348734117644),
 ('attendants', 0.7779384259487351),
 ('mastercard', 0.7662095992624447),
 ('orient', 0.762854944275653),
 ('pan', 0.7615831679026548),
 ('crashed', 0.7597347711139897),
 ('soil', 0.75524807332679)]

In [ ]:
glove2001.most_similar("firefighter", number= 10)

[('orleans', 0.9427751199967809),
 ('breed', 0.9392704766035931),
 ('mets', 0.9382023993246952),
 ('jersey', 0.9319797875477661),
 ('zealand', 0.93179093673504),
 ('yorkers', 0.9311996525727985),
 ('runways', 0.929329511918161),
 ('york', 0.9280836525071976),
 ('yorker', 0.9242927600848556)]

In [ ]:
glove2001.most_similar("afghanistan", number= 10)

[('fighting', 0.8364228034403239),
 ('taliban', 0.833003692708806),
 ('invade', 0.8075348622289268),
 ('regime', 0.8022704022216706),
 ('afghan', 0.7583552350981267),
 ('sudan', 0.7486523767710589),
 ('rebels', 0.7251282983800847),
 ('ayman', 0.716537188821748),
 ('prisoners', 0.7112868371677333)]

In [ ]:
glove2001.most_similar("islam", number= 10)

[('disney', 0.7191461086937099),
 ('veteran', 0.7059159134669828),
 ('waging', 0.7022037642418762),
 ('walt', 0.68459123281781),
 ('imf', 0.6758946923305603),
 ('walleyes', 0.6725917046833085),
 ('war', 0.6692598705284158),
 ('bundt', 0.660251739782124),
 ('fought', 0.6499430261223567)]

In [ ]:
glove2001.most_similar("jihad", number= 10)

[('islamic', 0.8908593596101719),
 ('militants', 0.8524939665927298),
 ('fundamentalism', 0.8153674119465896),
 ('radical', 0.7922016098152515),
 ('maker', 0.7827272040891483),
 ('fundamentalist', 0.7782159508817253),
 ('militia', 0.7397347847267481),
 ('egyptian', 0.739719535290954),
 ('hedonism', 0.7335185458463807)]

# Post 2001

## Preprocessing

In [ ]:
from word_forms.word_forms import get_word_forms
import nltk
nltk.download('omw-1.4')

# generate all possible forms of keywords
# keyword_list = ['bin laden', 'al qaeda', 'world trade center', 'twin towers', 'attack', 'hijack', 'firefighter', 'plane', 'terrorism', 'afghanistan', 'jihad', 'islam']
word_form_tower = get_word_forms("tower")
word_form_attack = get_word_forms("attack")
word_form_hijack = get_word_forms("hijack")
word_form_firefighter = get_word_forms("firefighter")
word_form_plane = get_word_forms("plane")
word_form_terrorism = get_word_forms("terrorism")


word_form_tot = (list(list(word_form_tower['a']) + list(word_form_tower['n']) + 
list(word_form_tower['v']) + list(word_form_tower['r']) + 
list(word_form_attack['a']) + list(word_form_attack['n']) + 
list(word_form_attack['v']) + list(word_form_attack['r']) + 
list(word_form_plane['a']) + list(word_form_plane['n']) + 
list(word_form_plane['v']) + list(word_form_plane['r']) + 
list(word_form_hijack['a']) + list(word_form_hijack['n']) + 
list(word_form_hijack['v']) + list(word_form_hijack['r']) + 
list(word_form_firefighter['a']) + list(word_form_firefighter['n']) + 
list(word_form_firefighter['v']) + list(word_form_firefighter['r']) + 
list(word_form_terrorism['a']) + list(word_form_terrorism['n']) + 
list(word_form_terrorism['v']) + list(word_form_terrorism['r'])))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
import string
import re

BASE_PATH = '/content/drive/MyDrive/ProgettiDataScience-SAD/1 ANNO 2 SEMESTRE/DATA SEMANTICS/Progetto/Newspaper/'

with open(BASE_PATH + 'text_news_nne/text_news_2002.txt') as read_file1:
  sentences2002 = read_file1.readlines()
with open(BASE_PATH + 'text_news_nne/text_news_2003.txt') as read_file2:
  sentences2003 = read_file2.readlines()
with open(BASE_PATH + 'text_news_nne/text_news_2004.txt') as read_file3:
  sentences2004 = read_file3.readlines()
with open(BASE_PATH + 'text_news_nne/text_news_2005.txt') as read_file4:
  sentences2005 = read_file4.readlines()
with open(BASE_PATH + 'text_news_nne/text_news_2006.txt') as read_file5:
  sentences2006 = read_file5.readlines()

sentences0206 = sentences2002 + sentences2003 + sentences2004 + sentences2005 + sentences2006

#define the list of words of our interest
keywords = ['bin laden', 'al qaeda', 'twin towers', 'world trade center', 'afghanistan', 'jihad', 'islam'] #aggiungere twin towers

keyword_list = keywords + word_form_tot

#define the dictionary where we store the articles
articles = {}
art = {}
#remove the p that represent in html the start of the paragraph
for symbol in sentences0206:
  symbol = symbol.replace('<p>', '')
  #selection of articles that contain the keywords
  for keyword in keyword_list:
    if (keyword in symbol.lower()):
      #select id of the article that contains at least one kwyword 
      id = re.findall(r'@@(\w+)', symbol)[0]
      articles[id] = symbol.lower()
      art[id] = symbol.lower()

print(len(articles))

8244


In [ ]:
import spacy
sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words

cont = 0
support_dict = {}

for id in art:
  #tokenization
  token_sentences0206_art = art[id].split()

  #stop-word removal
  stop_word_art = [i for i in token_sentences0206_art if str(i).lower() not in all_stopwords]
  words_art = [i for i in stop_word_art if i.isalpha() or i == "."]
  
  art[id] = words_art

  # bigrams = list(nltk.ngrams(words_art,2))
  # trigrams = list(nltk.ngrams(words_art,3))
  # n_most_common = 0 #how many of the most common n-grams do you want, zero indicates only the user_grams.
  # fdist_bigrams = nltk.FreqDist(bigrams).most_common(n_most_common) #n most common bigrams
  # fdist_trigrams = nltk.FreqDist(trigrams).most_common(n_most_common) #n most common trigrams
  # fdist_bigrams=[x[0][0]+' '+x[0][1] for x in fdist_bigrams]
  # fdist_trigrams=[x[0][0]+' '+x[0][1]+' '+x[0][2] for x in fdist_trigrams]
  # n_grams=fdist_bigrams
  # n_grams.extend(fdist_trigrams)

  user_grams = ['bin laden', 'al qaeda', 'world trade center', 'twin towers', 'white house', 'president bush', 'bay area']

  i=0
  support_dict[id] = []
  #check if 
  # - word in position i + space + word in position i+1 + space + word in position i+2 is a word in user_grams (world trade center)
  # - word in position i + space + word in position i+1 + space is a word in user_grams (ex. bin + ' ' + laden)
  # and create a unique term in the vocabulary (ex. bin-laden)
  # otherwise leave the single word 
  while i < len(art[id])-1:
    try:
      if (art[id][i] + ' ' + art[id][i+1] + ' ' + art[id][i+2] in user_grams):
        support_dict[id].append(art[id][i] + '-' + art[id][i+1] + '-' + art[id][i+2])
        i = i + 3
      elif(art[id][i] + ' ' + art[id][i+1] in user_grams):
        support_dict[id].append(art[id][i] + '-' + art[id][i+1])
        i = i + 2
      else:
        support_dict[id].append(art[id][i])
        i = i + 1
    except:
      if(art[id][i] + ' ' + art[id][i+1] in user_grams):
        support_dict[id].append(art[id][i] + '-' + art[id][i+1])
        i = i + 2
      else:
        support_dict[id].append(art[id][i])
        i = i + 1
      pass
    # print(i, 'out of', len(art[id]))

# just to know how many articles have been checked
  cont = cont + 1
  # print(cont)

In [ ]:
def split(sequence, sep):
    chunk = []
    for val in sequence:
        if val == sep:
            yield chunk
            chunk = []
        else:
            chunk.append(val)
    yield chunk

In [ ]:
# support_dict['3049580']

In [ ]:
# res = split(support_dict['3049580'], '.')

In [ ]:
# for x in res:
#   print(x)

In [ ]:
listGloVe0206 = []
for id in support_dict:
  res = split(support_dict[id], '.')
  for x in res:
    listGloVe0206.append(x)
  # listGloVe0206.remove([])

print(len(listGloVe0206))

768594


In [ ]:
listGloVe0206.remove([])

## GloVe

In [ ]:
from glove import Glove, Corpus
# from glove import *

# from glove_pybind import *
# from Glove import *


#Creating a corpus object
corpus = Corpus() # dictionary = dictionary2001 doesn't work

# lines = []
# for i in range(0, 1475):
#   lines.append(t[i][1])


#Training the corpus to generate the co-occurrence matrix which is used in GloVe
corpus.fit(listGloVe0206, window=3)

glove0206 = Glove() 
glove0206.fit(corpus.matrix, epochs=40, no_threads=4, verbose=True)
glove0206.add_dictionary(corpus.dictionary)
glove0206.save('glove0206.model')

Performing 40 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39


### Most similar to

In [ ]:
glove0206.most_similar("al-qaeda", number= 10)

[('taliban', 0.8659964653333638),
 ('blackmailed', 0.7917279757085314),
 ('fighters', 0.7849677181249263),
 ('qaeda', 0.743620793883025),
 ('fugitives', 0.7432920616245714),
 ('operatives', 0.729137022701392),
 ('dictator', 0.7009774419694538),
 ('alleged', 0.6948196907337216),
 ('zarqawi', 0.6899915941131678)]

In [ ]:
glove0206.most_similar("bin-laden", number= 10)

[('osama', 0.9603957171637726),
 ('lah', 0.8818389498013735),
 ('ayman', 0.7688495189488036),
 ('kha', 0.6975508795800677),
 ('cruze', 0.6815220918620395),
 ('battistel', 0.6727068814623403),
 ('hazelnuts', 0.6668739260114142),
 ('nalen', 0.6492032326679776),
 ('selleck', 0.6454372073418667)]

In [ ]:
glove0206.most_similar("airplane", number= 10)

[('airplanes', 0.7041537456505319),
 ('flying', 0.6964428855207268),
 ('plane', 0.6942348237370325),
 ('parked', 0.6891546914132187),
 ('grek', 0.6510984970451856),
 ('riding', 0.6354959540202757),
 ('uncheck', 0.6304703218373217),
 ('vacant', 0.6254399034853168),
 ('lots', 0.6236931819535213)]

In [ ]:
glove0206.most_similar("twin-towers", number= 10)

[('unbuilding', 0.7643750702175421),
 ('mushfiq', 0.7356715502199456),
 ('stieg', 0.6960734299167836),
 ('carnage', 0.6880452536470014),
 ('polymorphism', 0.6828607764765496),
 ('beguiled', 0.6786804854974379),
 ('andressa', 0.6738065990669936),
 ('nadolski', 0.670432020543623),
 ('faint', 0.6643451391642985)]

In [ ]:
glove0206.most_similar("tower", number= 10)

[('eiffel', 0.875653160913917),
 ('coit', 0.8491644122298109),
 ('moehler', 0.6513464378383642),
 ('occupied', 0.6490135793342835),
 ('destito', 0.647646270707761),
 ('dubill', 0.6374390539017482),
 ('defoyd', 0.627253319388888),
 ('anjuan', 0.6239757420452341),
 ('located', 0.6232890547063352)]

In [ ]:
glove0206.most_similar("terrorism", number= 10)

[('war', 0.8001724175902022),
 ('terror', 0.7870224865661218),
 ('peloponnesian', 0.7788700267438491),
 ('fight', 0.727437673826134),
 ('unwinnable', 0.7240733622407163),
 ('threat', 0.7122377451863948),
 ('unpopular', 0.7091869803496327),
 ('chaos', 0.695984363451062),
 ('criminals', 0.6922920898236001)]

In [ ]:
glove0206.most_similar("world-trade-center", number= 10)

[('bombing', 0.7904281540714131),
 ('unbuilding', 0.756740177774945),
 ('attack', 0.7152968884611688),
 ('bioterror', 0.7150365964027947),
 ('clones', 0.707609288127708),
 ('attacks', 0.6750804803877754),
 ('bioterrorist', 0.6740622304799656),
 ('assisted', 0.6729954468549824),
 ('fends', 0.657168079002831)]

In [ ]:
glove0206.most_similar("attack", number= 10)

[('attacks', 0.8992217480749248),
 ('clones', 0.8180103314105197),
 ('heart', 0.7844281314295445),
 ('terrorist', 0.7718004885729588),
 ('bioterrorist', 0.7444183748171319),
 ('prevent', 0.7287099666128274),
 ('failure', 0.7252814256525472),
 ('ied', 0.7251103486464456),
 ('bioterror', 0.7195737104315479)]

In [ ]:
glove0206.most_similar("hijacked", number= 10)

# 'hijacker',
#  'hijackings',
#  'hijackers',
#  'hijack',
#  'hijacks',
#  'hijacking',
#  'hijacked',
#  'hijack',
#  'hijacks',
#  'hijacking'

[('planes', 0.8596332451931789),
 ('crashed', 0.7736841921387764),
 ('dutchman', 0.7651851861752555),
 ('piloted', 0.7240609385470774),
 ('aircraft', 0.7033284515672613),
 ('fighter', 0.697173198887727),
 ('sullan', 0.6835496942028108),
 ('flying', 0.6725752165314322),
 ('helicopters', 0.6702289000515246)]

In [ ]:
glove0206.most_similar("firefighter", number= 10)

[('upstate', 0.894365837087767),
 ('yorker', 0.8486267474648104),
 ('islanders', 0.8400417198164083),
 ('zealander', 0.8356508545056499),
 ('auckland', 0.835056774213424),
 ('york', 0.8247853606268916),
 ('jets', 0.8164563500289087),
 ('rochelle', 0.8155216999252979),
 ('orleanians', 0.815295827577792)]

In [ ]:
glove0206.most_similar("afghanistan", number= 10)

[('iraq', 0.8702230845319232),
 ('invasion', 0.8470618160204745),
 ('taliban', 0.827181471178718),
 ('shuaib', 0.8065927753147408),
 ('occupation', 0.7900452492198912),
 ('kabul', 0.7889591657333846),
 ('invading', 0.7841800436134606),
 ('pakistan', 0.7735403236794113),
 ('postwar', 0.7632357411022392)]

In [ ]:
glove0206.most_similar("islam", number= 10)

[('shahara', 0.8780347236680389),
 ('darul', 0.8080852729268349),
 ('radical', 0.7661917397940428),
 ('holiest', 0.7287894583490488),
 ('rmc', 0.7052986100850519),
 ('shrines', 0.6821970601701742),
 ('christianity', 0.6805806076993245),
 ('muslim', 0.6756608006856919),
 ('mundyr', 0.6481427374955812)]

In [ ]:
glove0206.most_similar("jihad", number= 10)

[('islamic', 0.8658626898868566),
 ('extremists', 0.8093150835023429),
 ('militancy', 0.7804175024870744),
 ('tawhid', 0.7625184920953274),
 ('fundamentalism', 0.7601678452426444),
 ('hamas', 0.7598102318045684),
 ('extremist', 0.7525173669404188),
 ('islamist', 0.744438147586593),
 ('radicals', 0.7358369892451895)]

# **USELESS** - Prova Pre 2001

In [ ]:
BASE_PATH = '/content/drive/MyDrive/ProgettiDataScience-SAD/1 ANNO 2 SEMESTRE/DATA SEMANTICS/Progetto/Newspaper/'

def load_dict_from_file():
    f = open(BASE_PATH + 'dict9600.txt','r')
    data=f.read()
    f.close()
    return eval(data)

dictionary9600 = load_dict_from_file()

In [ ]:
t2 = []
for k, v in dictionary9600.items():
    t2.append((k,v))

In [ ]:
len(t2)

In [ ]:
from glove import Glove, Corpus
# from glove import *

# from glove_pybind import *
# from Glove import *


#Creating a corpus object
corpus9600 = Corpus() # dictionary = dictionary2001 doesn't work

lines = []
for i in range(0, 7958):
  lines.append(t2[i][1])

#Training the corpus to generate the co-occurrence matrix which is used in GloVe
corpus9600.fit(lines, window=3)

glove9600 = Glove() 
glove9600.fit(corpus9600.matrix, epochs=40, no_threads=4, verbose=True)
glove9600.add_dictionary(corpus9600.dictionary)
glove9600.save('glove9600.model')

In [ ]:
glove9600.most_similar('twin-towers', number = 10)

## Vedere cosa sono

In [ ]:
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [ ]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))

In [ ]:
print(find_closest_embeddings(embeddings_dict["king"])[1:6])

### TSNE

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
words =  list(embeddings_dict.keys())
vectors = [embeddings_dict[word] for word in words]
Y = tsne.fit_transform(vectors[:1000])
plt.scatter(Y[:, 0], Y[:, 1])

for label, x, y in zip(words, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords="offset points")
plt.show()